In [ ]:
from hmmlearn.hmm import GaussianHMM
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
data_fama_french = pd.read_csv('data/F-F_Research_Data_5_Factors_2x3.csv', skiprows=3)
data_stocks = pd.read_csv('/data/characteristics_data_feb2017.csv')

In [ ]:
data_fama_french['Date'] = pd.to_datetime(data_fama_french['Date'], format='%Y%m')
data_fama_french = data_fama_french.set_index('Date')

data_fama_french[['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'RF']] /= 100
data_fama_french

,Mkt-RF,SMB,HML,RMW,CMA,RF
Date,,,,,,
1963-07-01,-0.0039,-0.0041,-0.0097,0.0068,-0.0118,0.0027
1963-08-01,0.0507,-0.0080,0.0180,0.0036,-0.0035,0.0025
1963-09-01,-0.0157,-0.0052,0.0013,-0.0071,0.0029,0.0027
1963-10-01,0.0253,-0.0139,-0.0010,0.0280,-0.0201,0.0029
1963-11-01,-0.0085,-0.0088,0.0175,-0.0051,0.0224,0.0027
...,...,...,...,...,...,...
2024-08-01,0.0161,-0.0365,-0.0113,0.0085,0.0086,0.0048
2024-09-01,0.0174,-0.0102,-0.0259,0.0004,-0.0026,0.0040
2024-10-01,-0.0097,-0.0088,0.0089,-0.0138,0.0103,0.0039


In [ ]:
data_stocks.drop(
    columns=[
        "q10",
        "q20",
        "q50",
        "prc",
        "a2me",
        "ato",
        "beme",
        "c",
        "cto",
        "d2a",
        "dpi2a",
        "e2p",
        "fc2y",
        "free_cf",
        "investment",
        "lturnover",
        "noa",
        "oa",
        "ol",
        "pcm",
        "pm",
        "prof",
        "q",
        "rna",
        "roa",
        "roe",
        "s2p",
        "sga2m",
        "at",
        "cum_return_12_2",
        "cum_return_12_7",
        "cum_return_1_0",
        "cum_return_36_13",
        "idio_vol",
        "spread_mean",
        "suv",
        "rel_to_high_price",
        "lev",
    ],
    inplace=True,
)

data_stocks.sort_values(by=["date"], ascending=[True], inplace=True)
data_stocks["date"] = pd.to_datetime(data_stocks["date"])
data_stocks.head()